In [94]:
import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import re

In [95]:
import requests
import urllib.request
import pandas as pd
import csv
from bs4 import BeautifulSoup

In [96]:
df = pd.read_csv(os.path.abspath('../data/Processed/Cities_imputed.csv'), index_col=0)

In [97]:
climate_df = df[['City', 'Country', 'Continent']]

In [98]:
climate_df['wiki_link'] = [re.sub(r'\([^)]*\)', '', city).replace(" ", "_") for city in climate_df.City]

In [99]:
climate_df.loc[climate_df['wiki_link'] == 'Birmingham']

,City,Country,Continent,wiki_link
319,Birmingham(AL),United States,North America,Birmingham
261,Birmingham,United Kingdom,Europe,Birmingham


In [100]:
climate_df.loc[climate_df.index == 278] 

,City,Country,Continent,wiki_link
278,San Francisco Bay Area(CA),United States,North America,San_Francisco_Bay_Area


In [101]:
def update_wiki_link(df, city, city_update):
    df.loc[df['wiki_link'] == city, 'wiki_link'] = city_update

In [102]:
update_wiki_link(climate_df, 'Santa_Cruz', 'Santa_Cruz_(Bolivia)')

In [103]:
cities_updates = [
    ('Santa_Cruz', 'Santa_Cruz_(Bolivia)'),
    ('Cordoba' , 'Córdoba,_Argentina'),
    ('San_Francisco_Bay_Area', 'San_Fransisco'),
    ('Tampa-St._Petersburg', 'Saint_Petersburg_(Florida)'),
    ('Memphis', 'Memphis,_Tennessee'),
    ('Denver-Aurora', 'Aurora,_Colorado'),
    ('Rotterdam-Hague', 'Rotterdam'),
    ('Nashville-Davidson', 'Nashville,_Tennessee')
]

for city, city_update in cities_updates:
    update_wiki_link(
        df = climate_df,
        city = city,
        city_update = city_update
    )

    ('Birmingham', 'Birmingham,_Alabama')
    ('Valencia', 'Valencia,_Carabobo'),

In [104]:
climate_df.loc[(climate_df['wiki_link'] == 'Birmingham') & (climate_df['Country'] == 'United States')] = 'Birmingham,_Alabama'
climate_df.loc[(climate_df['wiki_link'] == 'Valencia') & (climate_df['Country'] == 'United Venezuela')] = 'Valencia,_Carabobo'

In [105]:
def seperate_metrix_from_imperial(row, th_data):
    rainfall = ['Average precipitation mm (inches)', 'Average rainfall inches (mm)', 'Average rainy days', 'Average precipitation inches (mm)', 'Average rainfall mm (inches)']

    matric = []
    for imperial_metric in row:
        try:
            val1, val2 = imperial_metric.replace(")", "").split('(')
            
            # Remove symbol which look like a minus sign which is not not a minus sign
            val1, val2 = val1.replace('−', '-'), val2.replace('−', '-')
            # Remove commas which seperates tree zeros and convert to float 
            val1, val2 = float(val1.replace(',', '')), float(val2.replace(',', ''))

            # if rainfall when select minimeters
            if th_data in rainfall:
                matric.append(max(val1, val2))
            # else it is a temperature select celcius
            else:
                matric.append(min(val1, val2))
        except ValueError:
            try:
                matric.append(float(imperial_metric))
            except ValueError:
                matric.append(None)
    return matric

In [106]:
focus_climate_data = [
    'Record high °C (°F)', 'Record high °F (°C)',
    'Average high °C (°F)', 'Average high °F (°C)', 
    'Daily mean °C (°F)', 'Daily mean °F (°C)', 'Daily mean °C (°F)',
    'Average low °C (°F)', 'Average low °F (°C)',
    'Record low °C (°F)', 'Record low °F (°C)',
    'Average precipitation mm (inches)', 'Average rainfall inches (mm)', 'Average rainy days', 'Average precipitation inches (mm)', 'Average rainfall mm (inches)',
    'Average precipitation days (≥ 0.1 mm)', 'Average rainy days (≥ 0.01 in)', 'Average precipitation days (≥ 1.0 mm)', 'Average precipitation days (≥ 0.5 mm)', 'Average precipitation days (≥ 0.01 in)', 'Average rainy days (≥ 0.1 mm)'
]

climate_data_names = [
    'Record high °C (°F)', 
    'Average high °C (°F)', 
    'Daily mean °C (°F)',
    'Average low °C (°F)', 
    'Record low °C (°F)', 
    'Average precipitation mm (inches)', 
    'Average precipitation days (≥ 0.1 mm)',
]

In [107]:
climate_data_dict = dict()

for wiki_link in climate_df.wiki_link:
    url = 'https://en.wikipedia.org/wiki/' + wiki_link

    response = requests.get(url)
    status_code = response.status_code
    print(wiki_link, '', status_code)

    soup = BeautifulSoup(response.text,"html.parser")

    # exstract the table with the cliamte data
    tables = soup.findAll('table',{"class":"collapsible"})
    for table in tables:
        if 'climate' in tables[0].findAll(['th'])[0].getText()[0:12].lower():
            break
    values = []
    trs = table.findAll(['tr'])[1:]


    for i, cell in enumerate(trs):

        th = cell.find_all('th')
        th_data = [col.text.strip('\n') for col in th]  
        td = cell.find_all('td')
        row = [i.text.replace('\n','') for i in td]
        
        
        if i == 0:
            columns = th_data
            city_climate_df_tmp = pd.DataFrame(columns=columns)
        else:
            
            if len(th_data) == 0:
                continue
            
            # Exstract the metric represeantion of the climate data
            metric_row = seperate_metrix_from_imperial(row, th_data[0])

            tmp = pd.DataFrame({key: [value] for key, value in zip(columns, th_data+metric_row)})
            city_climate_df_tmp = city_climate_df_tmp.append(tmp)

    climate_data_dict[wiki_link] = city_climate_df_tmp
    features = climate_data_dict[wiki_link]['Month'].values
    print(len(features), '  ', features)


Baltimore  200
9    ['Average high °F (°C)' 'Average low °F (°C)'
 'Average precipitation inches (mm)' 'Average snowfall inches (cm)'
 'Average precipitation days (≥ 0.01 in)' 'Average snowy days (≥ 0.1 in)'
 'Average relative humidity (%)' 'Mean monthly sunshine hours'
 'Percent possible sunshine']
Melbourne  200
9    ['Record high °C (°F)' 'Average high °C (°F)' 'Daily mean °C (°F)'
 'Average low °C (°F)' 'Record low °C (°F)' 'Average rainfall mm (inches)'
 'Average rainy days (≥ 1mm)' 'Average afternoon relative humidity (%)'
 'Mean monthly sunshine hours']
Niamey  200
9    ['Record high °C (°F)' 'Average high °C (°F)' 'Daily mean °C (°F)'
 'Average low °C (°F)' 'Record low °C (°F)'
 'Average precipitation mm (inches)'
 'Average precipitation days (≥ 1.0 mm)' 'Average relative humidity (%)'
 'Mean monthly sunshine hours']
Hanoi  200
9    ['Record high °C (°F)' 'Average high °C (°F)' 'Daily mean °C (°F)'
 'Average low °C (°F)' 'Record low °C (°F)' 'Average rainfall mm (inches)'
 'Ave

KeyError: 'Month'

In [136]:
wiki_link = 'Austin'


response = requests.get(url)
status_code = response.status_code
print(status_code)

soup = BeautifulSoup(response.text,"html.parser")


200


In [143]:

def extract_climate_table(soup):
    # exstract the table with the cliamte data
    tables = soup.findAll('table',{"class":"collapsible"})

    for table in tables:
        header = table.findAll(['th'])[0].getText()[0:12].lower()
        if 'climate' in header:
            select_table = table
            break
        else:
            select_table = None
    
    return select_table

trs = table.findAll(['tr'])[1:]

In [142]:
select_table

<table class="wikitable collapsible collapsed" style="width:100%; text-align:center; line-height: 1.2em; margin:auto;">
<tbody><tr>
<th colspan="14">Climate data for <a href="/wiki/Camp_Mabry" title="Camp Mabry">Camp Mabry</a>, Austin, Texas (1981–2010 normals,<sup class="reference" id="cite_ref-87"><a href="#cite_note-87">[b]</a></sup> extremes 1891–present)<sup class="reference" id="cite_ref-88"><a href="#cite_note-88">[c]</a></sup>
</th></tr>
<tr>
<th scope="row">Month
</th>
<th scope="col">Jan
</th>
<th scope="col">Feb
</th>
<th scope="col">Mar
</th>
<th scope="col">Apr
</th>
<th scope="col">May
</th>
<th scope="col">Jun
</th>
<th scope="col">Jul
</th>
<th scope="col">Aug
</th>
<th scope="col">Sep
</th>
<th scope="col">Oct
</th>
<th scope="col">Nov
</th>
<th scope="col">Dec
</th>
<th scope="col" style="border-left-width:medium">Year
</th></tr>
<tr style="text-align: center;">
<th scope="row" style="height: 16px;">Record high °F (°C)
</th>
<td style="background: #FF4100; color:#0000

In [123]:
'climate' in tables[0].findAll(['th'])[0].getText()[0:12].lower()

False

'Racial compo'

In [124]:
table.findAll(['th'])[0].getText()[0:12].lower()

'racial compo'

In [111]:

for i, cell in enumerate(trs):

    th = cell.find_all('th')
    th_data = [col.text.strip('\n') for col in th]  
    td = cell.find_all('td')
    row = [i.text.replace('\n','') for i in td]
    
    
    if i == 0:
        columns = th_data
        city_climate_df_tmp = pd.DataFrame(columns=columns)
    else:
        
        # exstract the metrix represeantion of the climate data
        if len(th_data) == 0:
            continue
        if th_data[0] in focus_climate_data:
            metric_row = seperate_metrix_from_imperial(row, th_data[0])

            tmp = pd.DataFrame({key: [value] for key, value in zip(columns, th_data+metric_row)})
            city_climate_df_tmp = city_climate_df_tmp.append(tmp)

In [112]:
city_climate_df_tmp

""


In [89]:
'climate' in tables[0].findAll(['th'])[0].getText()[0:12].lower() 

True

In [87]:
tmp = "Messi is the best soccerplayer"

In [88]:
"soccer" in tmp

True

In [69]:
trs

[<tr>
 <td><a href="/wiki/White_Americans" title="White Americans">White</a></td>
 <td>63.2%</td>
 <td>65.5%</td>
 <td>73.8%</td>
 <td>75.7%</td>
 <td>80.1%
 </td></tr>,
 <tr>
 <td>—Non-Hispanic</td>
 <td>55.4%</td>
 <td>58.6%</td>
 <td>73.2%</td>
 <td>75.2%</td>
 <td>79.5%<sup class="reference" id="cite_ref-fifteen_104-0"><a href="#cite_note-fifteen-104">[101]</a></sup>
 </td></tr>,
 <tr>
 <td><a href="/wiki/African_Americans" title="African Americans">Black or African American</a></td>
 <td>27.9%</td>
 <td>28.6%</td>
 <td>24.3%</td>
 <td>23.3%</td>
 <td>19.6%
 </td></tr>,
 <tr>
 <td><a href="/wiki/Hispanic_and_Latino_Americans" title="Hispanic and Latino Americans">Hispanic or Latino</a> (of any race)</td>
 <td>10.4%</td>
 <td>9.0%</td>
 <td>0.9%</td>
 <td>0.8%</td>
 <td>0.6%<sup class="reference" id="cite_ref-fifteen_104-1"><a href="#cite_note-fifteen-104">[101]</a></sup>
 </td></tr>,
 <tr>
 <td><a href="/wiki/Asian_Americans" title="Asian Americans">Asian</a></td>
 <td>3.6%</td>
 <